In [ ]:
%run visualisation.ipynb
%run read_NBAPlayers_graph.ipynb
%run GirvanNewmanAlgorithms.ipynb

In [ ]:
g = igraph.Graph()
readVertices("/home/irek/community-detection/Scrapers/BasketballReferenceScraper/data/Players.csv",g,True)
readEdges("/home/irek/community-detection/Scrapers/BasketballReferenceScraper/data/Edges.csv",g, True)

In [ ]:
AllStars1920 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
Overlap = [-1 for i in range(0, 10*len(AllStars1920))]

In [ ]:
AllStars181920 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588]

In [ ]:
for player in AllStars1920:
    print(g.vs[player].attributes())

In [ ]:
AllStars1920Graph = g.subgraph(AllStars181920)
print(AllStars1920Graph.ecount())
print(AllStars1920Graph.vcount())
for vertex in AllStars1920Graph.vs:
    vertex["communities"] = [0]
plot(AllStars1920Graph, "AllStars1920.svg")
result = []

In [ ]:
for player in AllStars1920Graph.vs:
    print(player)
for edge in AllStars1920Graph.es:
    print(edge.tuple)
    print(edge.attributes())
    print(' ')

In [ ]:
edge_betweenness = AllStars1920Graph.edge_betweenness(directed=False, cutoff=None, weights=None)
max_eb = max(edge_betweenness)
print(max_eb)
vertex_betweenness = AllStars1920Graph.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)
max_vb = max(vertex_betweenness)
print(max_vb)

In [ ]:
edgesToDelete = [AllStars1920Graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
print(edgesToDelete)
result.append(edgesToDelete)
print(result)
AllStars1920Graph.delete_edges(edgesToDelete)

In [ ]:
vertexToCopy = vertex_betweenness.index(max_vb);
print(vertexToCopy)
incident = AllStars1920Graph.incident(vertexToCopy, mode=ALL)
print(incident)
AllStars1920Graph.add_vertices(1)
AllStars1920Graph.vs[AllStars1920Graph.vcount()-1]['name'] = AllStars1920Graph.vs[vertexToCopy]['name']
Overlap[AllStars1920Graph.vcount()-1] = vertexToCopy
print(Overlap)

In [ ]:
sum = 0
for edge in incident:
    print(AllStars1920Graph.es[edge]['weight'])
    sum += AllStars1920Graph.es[edge]['weight']

average = sum/len(incident)
print(sum)
print(average)
edges_to_delete = []
new_vertex_counter = AllStars1920Graph.vcount()-1
allSameWeight = 1
popWeight = -1
for edge in incident:
    weight = AllStars1920Graph.es[edge]['weight']
    if popWeight != weight and edge != incident[0]:
        allSameWeight = 0
    if(weight<average):
        edges_to_delete.append(edge)
        source = AllStars1920Graph.es[edge].source
        target = AllStars1920Graph.es[edge].target
        print([source, target])
        if(target == vertexToCopy):
            print([(source, new_vertex_counter)])
            AllStars1920Graph.add_edges([(source, new_vertex_counter)])
            AllStars1920Graph.es[AllStars1920Graph.ecount()-1]["weight"]=weight
        else:
            print([(new_vertex_counter, target)])
            AllStars1920Graph.add_edges([(new_vertex_counter, target)])
            AllStars1920Graph.es[AllStars1920Graph.ecount()-1]["weight"]=weight
    popWeight = weight
if allSameWeight == 1:
    print('All same weight!')
    edges_to_delete.append(edge)
    AllStars1920Graph.add_edges([(AllStars1920Graph.es[incident[0]].source, new_vertex_counter)])
    AllStars1920Graph.es[AllStars1920Graph.ecount()-1]["weight"]=weight

In [ ]:
AllStars1920Graph.delete_edges(edges_to_delete)
toAdd = [(vertexToCopy, vertexToCopy)]
for edge in edges_to_delete:
    toAdd.append([(AllStars1920Graph.es[edge].source,AllStars1920Graph.es[edge].target)])
result.append(toAdd)
incident2 = AllStars1920Graph.incident(vertexToCopy, mode=ALL)
print(incident2)
print(result)

In [ ]:
#resultGraph = AllStars1920Graph.copy()
#resultGraph.delete_edges(resultGraph.es)
print(resultGraph.ecount())
print(resultGraph.vcount())

In [ ]:
#print(AllStars1920Graph.components())
print(resultGraph.components())
    

In [ ]:
print(result)

In [ ]:
resultGraph = AllStars1920Graph.copy()

In [ ]:
#print(result)
x = result[len(result)-2]
length = len(x)
print(length)
for edge in x:
    print(edge)
    resultGraph.add_edges([edge])

In [ ]:
comp = [[] for i in range(0, len(resultGraph.components()))]
counter = 0
for component in resultGraph.components():
    toAdd = []
    for vertex in component:
        if(vertex<39): #Change
            toAdd.append(vertex)
        else:
            while(Overlap[vertex]!=-1):
                vertex=Overlap[vertex]
            toAdd.append(vertex)
    comp[counter] = toAdd
    counter+=1
print(comp)

In [ ]:
resultToDraw = resultGraph.copy()

In [ ]:
counter = 0
for vertex in resultToDraw.vs:
    vertex["communities"] = []
for c in comp:
    for vertex in c:
        resultToDraw.vs[vertex]["communities"].append(counter)
        print(resultToDraw.vs[vertex]["communities"])
    counter+=1
print(resultToDraw.vcount())
plot_full(resultToDraw, "AllStars1920.svg")

In [ ]:
%run CONGAYearAlgorithm.ipynb

In [ ]:
AllStars1920 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
Overlap = [-1 for i in range(0, 10*len(AllStars1920))]
result = []
AllStars1920Graph = g.subgraph(AllStars1920)
print(AllStars1920Graph.ecount())
print(AllStars1920Graph.vcount())

In [ ]:
CONGAYearAlgorithm(AllStars1920Graph.copy(), result, Overlap)
print(result)
print(Overlap)

In [ ]:
AllStars181920 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588]
Overlap = [-1 for i in range(0, 10*len(AllStars181920))]
result = []
AllStars181920Graph = g.subgraph(AllStars181920)
print(AllStars181920Graph.ecount())
print(AllStars181920Graph.vcount())

In [ ]:
CONGAYearAlgorithm(AllStars181920Graph.copy(), result, Overlap)
print(result)
print(Overlap)